# Final Presentation ACDC

In this notebook we will walk through examples of this package. The following points are covered: 
- Find downstream vertices
- Find alternative edges
- Handle PGM input format
- Raise errors if the data is incorrect. 
- Aggregate the power flow results in tables. 
- Show tables with data of powerflow calculations. 
- Calculate EV penetration levels. 
- Calculate optimal tap positions. 
- Do N-1 Calculations. 

In [218]:
# some basic imports

import pytest
import graph_processing as tp  # Import power_system_simpulation.graphy_processing
import pandas as pd
import numpy as np
import scipy as sp

import power_flow_processing as pfp
from graph_processing import (
    EdgePairNotUniqueError,
    GraphCycleError,
    GraphNotFullyConnectedError,
    GraphProcessor,
    IDNotFoundError,
    IDNotUniqueError,
    InputLengthDoesNotMatchError,
)


# 1.1 Input dataset

We create an input graph by using the following parameters: 
- `vertex_ids`
- `edge_ids`
- `edge_vertex_id_pairs`
- `edge_enabled`
- `source_vertex_id`

In [219]:
vertex_ids = [0, 2, 4, 6, 10]  # All unique vertex ids
edge_ids = [1, 3, 5, 7, 8, 9]  # All unique edge ids
edge_vertex_id_pairs = [ # Which vertex ids are connected by an edge
        (0, 2),  # edge 1
        (0, 4),  # edge 3
        (0, 6),  # edge 5
        (2, 4),  # edge 7
        (4, 6),  # edge 8
        (2, 10),  # edge 9
    ]
edge_enabled = [True, True, True, False, False, True]  # Whether each edge is enabled or disabled
source_vertex_id = 0  # ID of the source vertex

This graph will result in this visual representation: 

In [220]:
'''      
vertex_0 (source) --edge_1(enabled)-- vertex_2 --edge_9(enabled)-- vertex_10
                 |                               |
                 |                           edge_7(disabled)
                 |                               |
                 -----------edge_3(enabled)-- vertex_4
                 |                               |
                 |                           edge_8(disabled)
                 |                               |
                 -----------edge_5(enabled)-- vertex_6
'''

'      \nvertex_0 (source) --edge_1(enabled)-- vertex_2 --edge_9(enabled)-- vertex_10\n                 |                               |\n                 |                           edge_7(disabled)\n                 |                               |\n                 -----------edge_3(enabled)-- vertex_4\n                 |                               |\n                 |                           edge_8(disabled)\n                 |                               |\n                 -----------edge_5(enabled)-- vertex_6\n'

# 1.2 Validation
This graph is tested for the following conditions: 
1. `vertex_ids` and `edge_ids` should be unique.
    - This function compares the length of all `vertex_ids` to the set of `vertex_ids` and gives an error if they are not the same. It uses the same approach for `edge_ids`.
2. `edge_vertex_id_pairs` should have the same length as `edge_ids`.
    - This function compares the length of the list of `edge_vertex_id_pairs` and `edge_ids`.
3. `edge_vertex_id_pairs` should contain valid `vertex ids`.
    - Using a loop all `edge_vertex_id_pairs` are checked to also be valid `edge_ids`.
4. `edge_enabled` should have the same length as `edge_ids`.
    - The length of the `edge_enabled` list is compared to length of the `edge_ids` list. 
5. `source_vertex_id` should be a valid `vertex_ids`.
     - The `source_vertex_id` is checked to be part of `vertex_ids`.
6. The graph should not contain cycles.
    - An adjacency list is built and using depth first search cycles are detected. 
7. The graph should be fully connected.
    - Using depth fist search the length of all the visited vertex and the length of `vertex_ids` is compared. 
8. Multiple edges should not connect the same two `vertex_ids`. 
    - A list of `edge_vertex_id_pairs` is compared to a list of the set of `edge_vertex_id_pairs`.

This can be tested by calling the `tp.GraphProcessor` function: 

In [221]:
test2 = tp.GraphProcessor(
    vertex_ids=vertex_ids,
    edge_ids=edge_ids,
    edge_vertex_id_pairs=edge_vertex_id_pairs,
    edge_enabled=edge_enabled,
    source_vertex_id=source_vertex_id,
    )

# Example
When for example not all `vertex_ids` are unique the following error will be raised: 

In [222]:
with pytest.raises(IDNotUniqueError) as excinfo:
    GraphProcessor([1, 2, 3, 3, 5], [1, 2, 3], [(1, 2), (2, 3), (1, 5)], [True, True, True], 1)
assert str(excinfo.value) == "Vertex IDs are not unique"

In [223]:

test = tp.GraphProcessor(
        vertex_ids=vertex_ids,
        edge_ids=edge_ids,
        edge_vertex_id_pairs=edge_vertex_id_pairs,
        edge_enabled=edge_enabled,
        source_vertex_id=source_vertex_id,
    )

# 1.3 Find downstream vertices

 Given an `edge_id`, return all the vertices which are in the downstream of the edge, with respect to the source vertex. Including the downstream vertex of the edge itself!
 Only the `edge_enabled` are taken into account. 
 The function returns a list of all downstream `vertex_ids`. 

In [224]:
vertex_ids = [0, 2, 4]  # All unique vertex ids
edge_ids = [1, 3]  # All unique edge ids
edge_vertex_id_pairs = [(0, 2), (2, 4)]  # Egde 1 and egde 3
edge_enabled = [True, True]  # Whether each edge is enabled or disabled
source_vertex_id = 0  # ID of the source vertex

test3 = tp.GraphProcessor(
        vertex_ids=vertex_ids,
        edge_ids=edge_ids,
        edge_vertex_id_pairs=edge_vertex_id_pairs,
        edge_enabled=edge_enabled,
        source_vertex_id=source_vertex_id,
    )

downstream_vertices = test3.find_downstream_vertices(1)
print(downstream_vertices)

[2, 4]


# 1.4 Find alternative edges 
Given an enabled edge the following analysis is done: 
- If this edge is going to be disabled. 
- Which (currently disabled) edge can be enabled to ensure that the graph is again fully connected and acyclic?
- Return a list of all alternative edges.

Our example graph would return the following results:   
        Call find_alternative_edges with disabled_edge_id=1 will return [7]   
        Call find_alternative_edges with disabled_edge_id=3 will return [7, 8]   
        Call find_alternative_edges with disabled_edge_id=5 will return [8]   
        Call find_alternative_edges with disabled_edge_id=9 will return []   

This function can be used by using the `find_alternative_edges` function and giving an `edge_ids` as input. 

In [225]:
alternative_edges = test2.find_alternative_edges(1)
print("Alternative edge when disabling edge 1 is:", alternative_edges)

alternative_edges = test2.find_alternative_edges(3)
print("Alternative edge when disabling edge 3 are:", alternative_edges)

alternative_edges = test2.find_alternative_edges(5)
print("Alternative edge when disabling edge 5 are:", alternative_edges)

alternative_edges = test2.find_alternative_edges(9)
print("Alternative edge when disabling edge 9 are:", alternative_edges)

Alternative edge when disabling edge 1 is: [7]
Alternative edge when disabling edge 3 are: [7, 8]
Alternative edge when disabling edge 5 are: [8]
Alternative edge when disabling edge 9 are: []


# 2.1 Power Grid Model (PGM) Input

The following sections describe a power grid calculation module with the use of the `power-grid-model` library. Firstly, we must handle the following inputs:
- A power grid in PGM format
- A table containing active load profile of all the `sym_load` in the grid, with timestamps and load ids.
- A table containing ractive load profile of all the `sym_load` in the grid, with timestamps and load ids.

This data can be imported using the following code:

In [226]:
from power_grid_model.utils import json_deserialize_from_file

grid_data = json_deserialize_from_file("input_network_data.json")
active_power_profile = pd.read_parquet("active_power_profile.parquet")
reactive_power_profile = pd.read_parquet("reactive_power_profile.parquet")

The `grid_data` consists of several different tables containing information about different elements of the grid, like lines, nodes, sources and load information.

In [227]:
print(pd.DataFrame(grid_data['line']))
print(pd.DataFrame(grid_data['node']))
print(pd.DataFrame(grid_data['source']))
print(pd.DataFrame(grid_data['sym_load']))

   id  from_node  to_node  from_status  to_status    r1   x1       c1  tan1  \
0   5          1        2            1          1  0.25  0.2  0.00001   0.0   
1   6          2        3            1          1  0.25  0.2  0.00001   0.0   
2   7          3        4            1          1  0.25  0.2  0.00001   0.0   

   r0  x0  c0  tan0     i_n  
0 NaN NaN NaN   NaN  1000.0  
1 NaN NaN NaN   NaN  1000.0  
2 NaN NaN NaN   NaN  1000.0  
   id  u_rated
0   1  10500.0
1   2  10500.0
2   3  10500.0
3   4  10500.0
   id  node  status  u_ref  u_ref_angle           sk  rx_ratio  z01_ratio
0  11     1       1    1.0          NaN  200000000.0       NaN        NaN
   id  node  status  type  p_specified  q_specified
0   8     2       1     0          0.0          0.0
1   9     3       1     0          0.0          0.0
2  10     4       1     0          0.0          0.0


The `active_power_profile` and `reactive_power_profile` contain the power profiles for different loads, so they tell how much power should be supplied to for example a household or factory (loads) at what time.

In [228]:
print(active_power_profile)
display(reactive_power_profile)

Load_ID                         8              9              10
Timestamp                                                       
2024-01-01 00:00:00   97627.007855  430378.732745  205526.752143
2024-01-01 01:00:00   89766.365994 -152690.401322  291788.226133
2024-01-01 02:00:00 -124825.577475  783546.001564  927325.521002
2024-01-01 03:00:00 -233116.962348  583450.076165   57789.839506
2024-01-01 04:00:00  136089.122188  851193.276585 -857927.883604
2024-01-01 05:00:00 -825741.400597 -959563.205119  665239.691096
2024-01-01 06:00:00  556313.501900  740024.296494  957236.684466
2024-01-01 07:00:00  598317.128433  -77041.275494  561058.352573
2024-01-01 08:00:00 -763451.148262  279842.042655 -713293.425182
2024-01-01 09:00:00  889337.834099   43696.643500 -170676.120019


Load_ID,8,9,10
Timestamp,,,
2024-01-01 00:00:00,-470888.775791,548467.378868,-87699.335567
2024-01-01 01:00:00,136867.897737,-962420.399127,235270.994152
2024-01-01 02:00:00,224191.445445,233867.993750,887496.157029
2024-01-01 03:00:00,363640.598207,-280984.198852,-125936.092401
2024-01-01 04:00:00,395262.391855,-879549.056741,333533.430891
2024-01-01 05:00:00,341275.739236,-579234.877852,-742147.404690
2024-01-01 06:00:00,-369143.298152,-272578.458115,140393.540836
2024-01-01 07:00:00,-122796.973075,976747.676118,-795910.378504
2024-01-01 08:00:00,-582246.487810,-677380.964230,306216.650931


# 2.2 Constructing PGM using input data

Using the previously imported input data, a power grid model (PGM) can be constructed. Furthermore, validation is important in this step, and as such an error should be raised if the data is invalid. The PGM can be constructed by calling the class `pfp.PowerFlow`. 

In [229]:
pgm = pfp.PowerFlow(grid_data)

The `PowerFlow` class contains an initialization function where the PGM is constructed, and the data is validated using the following line of code, which is already present in the actual function. 

In [230]:
pfp.assert_valid_input_data(input_data=grid_data, symmetric=True, calculation_type=pfp.CalculationType.power_flow)

# 2.3 PGM Batch update dataset and power flow calculation

The batch update dataset includes multiple power profiles (active and reactive power) for various nodes in the grid over a specified period. Instead of updating the grid model for each individual timestamp or node separately, the entire set of updates is applied at once, allowing us to perform the power flow calculation for the entire period.

Within the `batch_powerflow` function, the power flow calculation is also performed. This is done using the Newton-Raphson method, which returns the `output_data` which contains the solution to the power flow calculation. Furthermore, the batch update dataset is also validated within the function.

This entire functionality can easily be performed with the following line of code:

In [231]:
output_data = pgm.batch_powerflow(active_power_profile, reactive_power_profile)

# 2.4 Aggregating Power Flow Results

Now all the data has been prepared and the state of the power grid after applying all batch updates has been computed, the power flow results will be aggregated in two tables:
- A voltage table with each row representing a timestamp and containing the following columns:
    - Timestamp (index column)
    - Maximum p.u. voltage of all the nodes for this timestamp
    - The node ID with the maximum p.u. voltage
    - Minimum p.u. voltage of all the nodes for this timestamp
    - The node ID with the minimum p.u. voltage

- A loading table with each row representing a line and containing the following columns:
    - Line ID (index column)
    - Total energy loss of the line in kWh over the entire period
    - Maximum loading in p.u. of the line across the whole timeline
    - Timestamp of this maximum loading moment
    - Minimum loading in p.u. of the line across the whole timeline
    - Timestamp of this minimum loading moment

These tables are constructed using two separate functions, which also gives the results for the given input data and power profiles:

In [232]:
pgm.aggregate_voltage_table(active_power_profile, reactive_power_profile)

,Max_Voltage,Max_Voltage_Node,Min_Voltage,Min_Voltage_Node
Timestamp,,,,
2024-01-01 00:00:00,1.004847,1,1.003450,3
2024-01-01 01:00:00,1.012053,3,1.007998,1
2024-01-01 02:00:00,0.997474,1,0.984365,4
2024-01-01 03:00:00,1.006557,4,1.005190,1
2024-01-01 04:00:00,1.011007,4,1.005877,1
2024-01-01 05:00:00,1.020486,4,1.010570,1
2024-01-01 06:00:00,1.006342,1,0.998868,4
2024-01-01 07:00:00,1.004306,1,1.002822,3
2024-01-01 08:00:00,1.020402,4,1.010501,1


In [233]:
pgm.aggregate_loading_table(active_power_profile, reactive_power_profile)

,Total_Loss,Max_Loading,Max_Loading_Timestamp,Min_Loading,Min_Loading_Timestamp
Line_ID,,,,,
5,63.294763,0.149830,2024-01-01 06:00:00,0.063798,2024-01-01 03:00:00
6,36.775143,0.111039,2024-01-01 05:00:00,0.037184,2024-01-01 00:00:00
7,14.872359,0.071700,2024-01-01 02:00:00,0.020380,2024-01-01 01:00:00


# 3.1 Using a LV grid with a MV/LV transformer

This part of the package will present low voltage grid analytics functions. These analytics include EV penetration level and optimal tap position. Input data can be given as: 
- A LV grid in PGM input format.
- LV feeder IDs list.
- A (active and reactive) load profile.
- A pool of EV charging profiles for the same time period as the time period of load profile.

The data is validated to check for the following criteria: 
- The LV grid should be a valid PGM input data.
- The LV grid has exactly one `transformer`, and one `source`.
- All IDs in the LV Feeder IDs are valid line IDs.
- All the lines in the LV Feeder IDs have the `from_node` the same as the `to_node` of the `transformer`.
- The grid is fully connected in the initial state.
- The grid has no cycles in the initial state.
- The timestamps are matching between the active load profile, reactive load profile, and EV charging profile.
- The IDs in active load profile and reactive load profile are matching.
- The IDs in active load profile and reactive load profile are valid IDs of `sym_load`.
- The number of EV charging profile is at least the same as the number of `sym_load`.

In [234]:
import os
import math
import pandas as pd
import graph_processing as tp  
from power_grid_model.utils import json_deserialize_from_file
import power_flow_processing as pfp
import power_system_simulation as pss

number_of_houses = 4
number_of_feeders = 2
penetration_level = 0.80
# total_evs = number_of_houses * penetration_level
# evs_per_feeder = math.floor(total_evs / number_of_feeders)
# print(f"EVs per feeder: {evs_per_feeder}")

# Determine the file path relative to this script's location
grid_data = json_deserialize_from_file("input_network_data_assign3.json")
active_power_profile = pd.read_parquet("active_power_profile_assign3.parquet")
ev_active_power_profile = pd.read_parquet("ev_active_power_profile.parquet")
reactive_power_profile = pd.read_parquet("reactive_power_profile_assign3.parquet")



EV_test = pss.PowerSim(grid_data=grid_data)

# print(ev_active_power_profile.columns)



    # Randomly select houses for EV assignment within each feeder
    # These values should be given as an input from another file
    # Within a LV feeder, randomly select houses which will have EVs
    # For each selected house with EV, randomly select an EV charging profile to add to the sym_load of that house.
    # After assignment of EV profiles, run a time-series power flow as in Assignment 2, return the two aggregation tables.

# edge_vertex_id_pairs = list(zip(grid_data["line"]["from_node"], grid_data["line"]["to_node"])) + list(
#             zip(grid_data["transformer"]["from_node"], grid_data["transformer"]["to_node"])
#         )
# edge_enabled = []
# for i in grid_data["line"]["id"]:
#             index = np.where(grid_data["line"]["id"] == i)
#             if grid_data["line"][index]["from_status"] == 1 & grid_data["line"][index]["to_status"] == 1:
#                 edge_enabled = edge_enabled + [True]
#             else:
#                 edge_enabled = edge_enabled + [False]
# if grid_data["transformer"][0]["from_status"] == 1 & grid_data["transformer"][0]["to_status"] == 1:
#             edge_enabled = edge_enabled + [True]
# else:
#             edge_enabled = edge_enabled + [False]
# source_vertex_id = grid_data["source"]["node"][0]
# edge_ids = list(grid_data["line"]["id"]) + list(grid_data["transformer"]["id"])
# vertex_ids = grid_data["node"]["id"]
# test4 = tp.GraphProcessor(
#         vertex_ids=vertex_ids,
#         edge_ids=edge_ids,
#         edge_vertex_id_pairs=edge_vertex_id_pairs,
#         edge_enabled=edge_enabled,
#         source_vertex_id=source_vertex_id,
#     )
# transformer_to_node = grid_data["transformer"][0]["to_node"]

# # assigned_profiles = set()   # This keeps track of the assigned profiles

# for line in grid_data["line"]:
#     if line["from_node"] == transformer_to_node:
#         print(f"Found line with from_node {transformer_to_node}: to_node = {line['to_node']}")
#         print(f"Edge details: id = {line['id']}, r1 = {line['r1']}, x1 = {line['x1']}, ...")  # Find all feeders
#         downstream_vertices = test4.find_downstream_vertices(line['id'])
#         #print(downstream_vertices)
#         sym_load_nodes = set(grid_data['sym_load']['node'])
#         common_nodes = sym_load_nodes.intersection(downstream_vertices)
#         print(common_nodes) # all the nodes in a downstream vertex that have a sym load
#         '''
#         # ---- asigment of profiles
#         ev_assignment_counter = 0
#         for node in common_nodes:
#             if ev_assignment_counter >= evs_per_feeder:
#                 break
#     for idx, row in ev_active_power_profile.iterrows():
#                 if idx not in assigned_profiles:
#                     # Assign EV profile to active_power_profile at the corresponding node
#                     active_power_profile.loc[active_power_profile['node'] == node, 'ev_profile'] = idx
#                     assigned_profiles.add(idx)
#                     ev_assignment_counter += 1
#                     break
#                 if ev_assignment_counter >= evs_per_feeder:
#                     break  # Stop further processing of lines once we've assigned enough EV profiles
#         '''


In [235]:
print(type(active_power_profile))
display(active_power_profile)

load_id_ev = 12

# print(active_power_profile[12].values)

<class 'pandas.core.frame.DataFrame'>


Load ID,12,13,14,15
Timestamp,,,,
2025-01-01 00:00:00,53845.936311,52868.809400,59437.379259,61193.999654
2025-01-01 00:15:00,58623.012703,66992.056154,57797.477123,48956.385676
2025-01-01 00:30:00,64800.016277,54713.600702,51293.354771,58092.153016
2025-01-01 00:45:00,61525.469129,59903.401304,49747.760990,62055.246675
2025-01-01 01:00:00,60289.133403,56883.894966,56301.650801,56226.510409
...,...,...,...,...
2025-01-10 22:45:00,69484.715910,71348.849463,47979.322152,64352.546069
2025-01-10 23:00:00,61503.570916,60836.586259,76135.394978,59164.472679
2025-01-10 23:15:00,56045.155706,54275.886219,63557.959847,66834.711500


[ 53845.93631081  58623.01270271  64800.01627673  61525.46912866
  60289.13340255  56455.46675888  65656.8226738   70279.3258901
  76672.32469814  70839.93097702  64291.62379938  66433.72945577
  81018.32166715  83365.0331567   58079.39515928  71313.05621048
  89547.75558582 102276.87982595 100565.89911725  99611.29447291
 112170.06590184  86333.0595454  100043.71849798  94775.61333107
  92020.85356511  83685.58990963  86570.85767516 104108.98462837
  99927.76582002  90081.75573189  80825.20989899  88175.87655796
  92959.04497331 117468.93961499  82030.0126709   82645.06682156
  83299.91446806  76083.86400862 101328.71481962 101042.62980977
  79048.74913169  91684.51032988 101875.72017127  94180.69262115
  98318.38582369  90720.79423517  90607.67746124  99241.10772537
 101442.16391237  93754.74249227 103650.17987328  94472.25537046
 100860.63516092  90603.17422822  73652.63340661  94586.11711276
 103274.17085012  79975.52101486  93441.606541    91325.12284175
  90564.84026318  75410.12

In [236]:
print(type(ev_active_power_profile))
display(ev_active_power_profile)

<class 'pandas.core.frame.DataFrame'>


EV Profile Sequence Number,0,1,2,3
Timestamp,,,,
2025-01-01 00:00:00,0.0,0.000000,0.0,0.0
2025-01-01 00:15:00,0.0,0.000000,0.0,0.0
2025-01-01 00:30:00,0.0,0.000000,0.0,0.0
2025-01-01 00:45:00,0.0,0.000000,0.0,0.0
2025-01-01 01:00:00,0.0,0.000000,0.0,0.0
...,...,...,...,...
2025-01-10 22:45:00,0.0,10984.380611,0.0,0.0
2025-01-10 23:00:00,0.0,10991.903421,0.0,0.0
2025-01-10 23:15:00,0.0,10995.014401,0.0,0.0


In [239]:
combined_profile = pd.DataFrame(active_power_profile[12].values + ev_active_power_profile[0].values)

display(combined_profile)

,0
0,53845.936311
1,58623.012703
2,64800.016277
3,61525.469129
4,60289.133403
...,...
955,69484.715910
956,61503.570916
957,56045.155706
958,73370.995338


In [238]:
result_ev = EV_test.ev_penetration(num_houses=number_of_houses, num_feeders=number_of_feeders, penetration_level=penetration_level, active_power_profile=active_power_profile, reactive_power_profile=reactive_power_profile, ev_active_power_profile=ev_active_power_profile)


EVs per feeder: 1
Found line with from_node 1: to_node = 2
Edge details: id = 16, r1 = 0.0003099357220775627, x1 = 0.0001406945054167883, ...
{3, 5}
Found line with from_node 1: to_node = 6
Edge details: id = 20, r1 = 0.0003164960265303119, x1 = 0.0001436725383591547, ...
{9, 7}
